<a href="https://colab.research.google.com/github/SangsooYoo/DHistory/blob/master/study/ML/3rd_live_class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!nvidia-smi

Fri Jun 25 10:18:59 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P8    29W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
import tensorflow as tf
from tensorflow import keras
from matplotlib import pyplot as ply

Transfer Learning을 통한 Cats and Dogs 데이터셋 분류 성능 개선

Step 1. input tensor와 target tensor준비

(1) Cats and Dogs 데이터셋 다운로드

In [6]:
url ='https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'

In [11]:
keras.utils.get_file(fname='cats_and_dogs_filtered.zip', 
                     origin=url, 
                     extract=True, 
                     cache_dir='/content')

'/content/datasets/cats_and_dogs_filtered.zip'

(2) ImageDataGenerator를 이용해 이미지 파일을 load하기위한 경로 지정

In [12]:
train_dir = '/content/datasets/cats_and_dogs_filtered/train'
validation_dir = '/content/datasets/cats_and_dogs_filtered/validation'

(3) ImageDataGenerator객체 생성

In [15]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_gen = ImageDataGenerator(rescale=1./255.)
validation_gen = ImageDataGenerator(rescale=1./255.) 

.flow_from_directory()메서드를 이용하여 학습데이터와 검증데이터를 위한 DirectoryIterator객체 생성

In [17]:
train_generator = train_gen.flow_from_directory(directory=train_dir, 
                              target_size = (224,224),
                              batch_size=32,
                              shuffle=True,
                              class_mode='binary')

Found 2000 images belonging to 2 classes.


In [22]:
x, y = next(train_generator)
x.shape

(32, 224, 224, 3)

In [23]:
y.shape

(32,)

In [24]:
y

array([0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 1., 0.,
       1., 1., 1., 0., 1., 0., 0., 1., 0., 1., 0., 1., 1., 0., 1.],
      dtype=float32)

In [25]:
validation_generator = train_gen.flow_from_directory(directory=validation_dir, 
                              target_size = (224,224),
                              batch_size=32,
                              class_mode='binary')

Found 1000 images belonging to 2 classes.


Step 2. VGG16을 Backbone으로 하는 모델 디자인 및 학습정보 설정
(1)Pre-trained된 VGG16모델 객체 생성


*   imagenet 데이터를 이용해 학습된 모델 객체생성
*   classification layer 제외



In [28]:
from tensorflow.keras.applications import VGG16

In [35]:
conv_base = VGG16(include_top=False, weights='imagenet', input_shape=(224, 224, 3))

In [36]:
model = keras.models.Sequential()
model.add(conv_base)
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(units=256, activation='relu'))
model.add(keras.layers.Dense(units=1, activation='sigmoid'))